# XGBoost
XGBoost is an optimized distributed gradient boosting library designed to be highly efficient, flexible and portable. It implements machine learning algorithms under the Gradient Boosting framework. XGBoost provides a parallel tree boosting (also known as GBDT, GBM) that solve many data science problems in a fast and accurate way. XGBoost was first added in MADlib 1.20.0.

In [ ]:
%load_ext sql

In [ ]:
# Greenplum Database 6.X
%sql postgresql://okislal@localhost:6600/madlib

In [ ]:
%sql select madlib.version();
#%sql select version();

# 1.  Load data

The sample data for XGBoost can be downloaded from the examples section of the MADlib documentation. Direct link: https://madlib.apache.org/docs/latest/example/madlib_xgboost_example.sql

In [ ]:
%%sql 
SELECT * FROM abalone LIMIT 10;

# 2. Run a single XGBoost training
Note that the function collates the data into a single segment and runs the xgboost python process on that machine.

In [ ]:
%%sql
DROP TABLE IF EXISTS xgb_single_out, xgb_single_out_summary;
SELECT madlib.xgboost(
    'abalone',  -- Training table
    'xgb_single_out',  -- Grid search results table.
    'id',       -- Id column
    'sex',      -- Class label column
    '*',        -- Independent variables  
    NULL,       -- Columns to exclude from features 
    $$ 
    {
        'learning_rate': [0.01], #Regularization on weights (eta). For smaller values, increase n_estimators
        'max_depth': [9],#Larger values could lead to overfitting
        'subsample': [0.85],#introduce randomness in samples picked to prevent overfitting
        'colsample_bytree': [0.85],#introduce randomness in features picked to prevent overfitting
        'min_child_weight': [10],#larger values will prevent over-fitting
        'n_estimators':[100] #More estimators, lesser variance (better fit on test set) 
    } 
    $$,         -- XGBoost grid search parameters
    '',         -- Sample weights
    0.8,        -- Training set size ratio
    NULL        -- Variable used to do the test/train split.
);

SELECT features, importance, precision, recall, fscore, support FROM xgb_single_out_summary;

# 3. Run XGBoost Prediction

In [ ]:
%%sql
DROP TABLE IF EXISTS xgb_single_score_out, xgb_single_score_out_metrics, xgb_single_score_out_roc_curve;

SELECT madlib.xgboost_predict(
    'abalone',          -- test_table
    'xgb_single_out',   -- model_table
    'xgb_single_score_out',    -- predict_output_table
    'id',               -- id_column
    'sex'               -- class_label
);

SELECT * FROM xgb_single_score_out LIMIT 10;

# 4. Run XGBoost with grid search
The parameter options are combined to form a grid and explored in parallel by running distinct xgboost processes in different segments in parallel. The following example will generate 4 configurations to test by combining 'learning_rate': [0.01,0.1] and 'max_depth': [9,12].

In [ ]:
%%sql
DROP TABLE IF EXISTS xgb_grid_out, xgb_grid_out_summary;

SELECT madlib.xgboost(
    'abalone',  -- Training table
    'xgb_grid_out',  -- Grid search results table.
    'id',       -- Id column
    'sex',      -- Class label column
    '*',        -- Independent variables
    NULL,       -- Columns to exclude from features
    $$
    {
        'learning_rate': [0.01,0.1], #Regularization on weights (eta). For smaller values, increase n_estimators
        'max_depth': [9,12],#Larger values could lead to overfitting
        'subsample': [0.85],#introduce randomness in samples picked to prevent overfitting
        'colsample_bytree': [0.85],#introduce randomness in features picked to prevent overfitting
        'min_child_weight': [10],#larger values will prevent over-fitting
        'n_estimators':[100] #More estimators, lesser variance (better fit on test set)
    }
    $$,         -- XGBoost grid search parameters
    '',         -- Sample weights
    0.8,        -- Training set size ratio
    NULL        -- Variable used to do the test/train split.
);

SELECT features, params, importance, precision, recall, fscore, support, params_index FROM xgb_grid_out_summary ORDER BY params_index;

# 5.  Run XGBoost Prediction on Grid Output Table
Let's say we are interested in the model 2 and want to run a prediction using it.

In [ ]:
%%sql

DROP TABLE IF EXISTS xgb_grid_score_out, xgb_grid_score_out_metrics, xgb_grid_score_out_roc_curve;

SELECT madlib.xgboost_predict(
    'abalone',               -- test_table
    'xgb_grid_out',          -- model_table
    'xgb_grid_score_out',    -- predict_output_table
    'id',                    -- id_column
    'sex',                   -- class_label
    2                        -- model_filters
);
SELECT * FROM xgb_grid_score_out LIMIT 10;